## Парсинг HTML. BeautifulSoup

В данном ноутбуке реализован парсинг основной информации по вакансиям с сайтов hh.ru и superjob.ru.
Пользователь может передать через input ключевые слова для поиска и регион (поиск работает для 12-ти крупных российских городов). Приложение анализирует все доступные страницы поиска.
Результат работы парсера сохраняется в \*.csv файл, содержащий следующие поля:

- ID вакансии
- Должность
- Работодатель
- Зарплата (отдельно минимум, максимум и валюта)
- Cсылка на вакансию
- Источник вакансии
- Описание вакансии

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
import pandas as pd

In [2]:
# параметры поиска
query = input('Введите запрос: ').lower()
area = input('Введите город для поиска: ').lower()

Введите запрос: прораб
Введите город для поиска: екатеринбург


In [3]:
# словарь для результатов поиска
vacancies = {
    'query': f'{query}_{area}',
    'results': []
}

In [4]:
# словари для трансформации региона поиска
hh_area_dict = {
    'москва': '1',
    'санкт-петербург': '2',
    'екатеринбург': '3',
    'новосибирск': '4',
    'нижний новгород': '66',
    'казань': '88',
    'воронеж': '26',
    'волгоград': '24',
    'ростов-на-дону': '76',
    'краснодар': '53',
    'уфа': '99', 
    'хабаровск': '102'
}

sj_area_dict = {
    'москва': 'москва',
    'санкт-петербург': 'spb',
    'екатеринбург': 'ekaterinburg',
    'новосибирск': 'nsk',
    'нижний новгород': 'nn',
    'казань': 'kazan',
    'воронеж': 'voronezh',
    'волгоград': 'volgograd',
    'ростов-на-дону': 'rnd',
    'краснодар': 'krasnodar',
    'уфа': 'ufa', 
    'хабаровск': 'habarovsk'
}

In [5]:
# преобразовываем input в понятный парсеру
def hh_area_transform(area):
    if area in hh_area_dict:
        return hh_area_dict[area]
    else:
        print('Неизвестный регион')
    
def sj_area_transform(area):
    if area in sj_area_dict:
        return sj_area_dict[area]
    else:
        print('Неизвестный регион')
        
hh_area = hh_area_transform(area)
sj_area = sj_area_transform(area)

In [6]:
hh = 'https://hh.ru/'
sj = 'https://www.superjob.ru'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:74.0) Gecko/20100101 Firefox/74.0'}

# задаем счетчики страниц - у hh - c 0, у sj - c 1
hh_page_count = range(100)
sj_page_count = range(1,100)

# создаем паттерны для извлечения id вакансии
hh_id_pattern = re.compile(r'https://[^/]+/vacancy/(\d+)')
sj_id_pattern = re.compile(r'https://www.superjob.ru/vakansii/[\w+\-\w+]+\-(\d+)')

**hh.ru parsing**

In [7]:
def get_title_hh(vacancy):
    return vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-title'})[0].getText()


def get_url_hh(vacancy):
    return vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-title'})[0]['href']
    

def get_id_hh(vacancy):
    return f'hh_{hh_id_pattern.findall(get_url_hh(vacancy))[0]}'


def get_employer_hh(vacancy):
    employer_el = vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
    # случай, когда работодатель указан описательно (нет ссылки на компанию)
    if employer_el:
        return employer_el[0].getText()
    else:
        return vacancy.findChildren('div', {'class': 'vacancy-serp-item__meta-info'})[0].getText() 

    
def get_salary_hh(vacancy):
    #  зарплата
    min_salary, max_salary, currency = None, None, None

    salary_el = vacancy.findChildren('span', {'data-qa': "vacancy-serp__vacancy-compensation"})

    if not salary_el:
        return min_salary, max_salary, currency
    
    # разбиваем по тире
    split_range = re.split('-', salary_el[0].getText())
    if len(split_range) == 2:
        min_salary = int(''.join(re.findall('\d+', split_range[0])))
        items = re.findall('\w+', split_range[1])
        max_salary = int(''.join(items[:-1]))
        currency = items[-1] 
    else:
        items = re.findall('\w+', split_range[0])
        if items[0] == 'от':
            min_salary = int(''.join(items[1:-1]))
        else:
            max_salary = int(''.join(items[1:-1]))         
        currency = items[-1]

    return min_salary, max_salary, currency


def get_description_hh(vacancy):
    desc1 = vacancy.findChildren('div', {'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'})[0].getText()
    desc2 = vacancy.findChildren('div', {'data-qa': 'vacancy-serp__vacancy_snippet_requirement'})[0].getText()
    return desc1 + ' ' + desc2

In [8]:
for i in hh_page_count:
    hh_link = f'{hh}search/vacancy?area={hh_area}&st=searchVacancy&text={query}&page={i}'
    hh_page = requests.get(hh_link, headers=headers)
    
    if not hh_page.ok:
        break
    
    # формируем список элементов страницы, содержащих вакансии
    hh_soup = bs(hh_page.text,'lxml')
    vacancies_block = hh_soup.find_all('div', {'class': 'vacancy-serp'})[0]
    vacancies_list = vacancies_block.find_all('div', {'class': 'vacancy-serp-item'})
    
    # определяем условие выхода из цикла (вакансий больше нет)
    if len(vacancies_list) == 0:
        break
    
    # итерируемся по списку, содержащем вакансии
    for vacancy in vacancies_list:
        salary = get_salary_hh(vacancy)

        vacancy_data = {
            'url': get_url_hh(vacancy),
            'id': get_id_hh(vacancy),
            'title': get_title_hh(vacancy),
            'employer': get_employer_hh(vacancy),
            'min_salary': salary[0],
            'max_salary': salary[1],
            'currency': salary[2],
            'source': hh,
            'short_desc': get_description_hh(vacancy)
        }
        
        # пополняем основной словарь вакансий
        vacancies['results'].append(vacancy_data)

**Superjob.ru parsing**

In [9]:
# записываем классы
vac_block = '_1Ttd8 _2CsQi'
vac_class = '_3zucV f-test-vacancy-item _3j3cA RwN9e _3tNK- _1NStQ _1I1pc'        
title_url_class = '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'
salary_class = '_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'
employer_class = '_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'
location = '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'
desc_class = '_3mfro _3V-Qt _9fXTd _2JVkc _2VHxz'

In [10]:
def get_title_sj(vacancy):
    title = vacancy.findChildren('div', {'class': title_url_class})
    # обрабатываем случай, когда title лежит не в span, а в h2
    if title:
        return title[0].getText()
    else:
        return vacancy.findChildren('h2', {'class': title_url_class})[0].getText()


def get_employer_sj(vacancy):
    employer_el = vacancy.findChildren('span', {'class':employer_class})
    # обрабатываем случай, когда работодатель не указан
    if employer_el:
        return employer_el[0].getText()
    else:
        return None

def get_salary_sj(vacancy):
    salary_el = vacancy.findChildren('span', {'class': salary_class})[0].getText()
    min_salary, max_salary, currency = None, None, None

    if salary_el != 'По договорённости':
        #разбиваем по тире, заменяем неразрывные пробелы
        split_range = re.split('—', salary_el.replace('\xa0', ' '))

        # диапазон
        if len(split_range) == 2:
            min_salary = int(''.join(re.findall('\d+', split_range[0])))
            items = re.findall('\w+', split_range[1])
            max_salary = int(''.join(items[:-1]))
            currency = items[-1] 
        else:
            # разбиваем по пробелам список с одним элементом
            items_one = re.findall('\w+', split_range[0])
            # от или до
            if items_one[0] == 'от':
                min_salary = int(''.join(items_one[1:-1]))
            elif items_one[0] == 'до':
                max_salary = int(''.join(items_one[1:-1]) ) 
            # зп одним числом
            else:
                min_salary = int(''.join(items_one[:-1])) 
                max_salary = min_salary
            currency = items_one[-1]

    return min_salary, max_salary, currency


def get_url_sj(vacancy):
    url = vacancy.findChildren('div', {'class': title_url_class})
    # обрабатываем случай, когда url лежит не в span, а в h2
    if url:
        url = url[0].find('a')['href']
        return sj + url             
    else:
        url = vacancy.findChildren('h2', {'class': title_url_class})[0].find('a')['href']
        return sj + url
    
    
def get_id_sj(vacancy):
    return f"sj_{sj_id_pattern.findall(get_url_sj(vacancy))[0]}"


def get_description_sj(vacancy):
    desc_el = vacancy.findChildren('span', {'class': desc_class})
    # обрабатываем случай, когда описания вакансии нет
    if desc_el:
        return desc_el[0].getText()
    else:
        return None

In [11]:
for i in sj_page_count:
    # обрабатываем случай, когда регион 'москва'
    if sj_area == 'москва':
        sj_link = f'{sj}/vacancy/search/?keywords={query}&geo[t][0]=4&page={i}'
    else:
        sj_link = f'https://{sj_area}.superjob.ru/vacancy/search/?keywords={query}&page={i}'
    sj_page = requests.get(sj_link, headers=headers)

    if not sj_page.ok:
        break
        
    # формируем список элементов страницы, содержащих вакансии
    sj_soup = bs(sj_page.text,'lxml')
    vacancies_block = sj_soup.find_all('div', {'class': vac_block})[0]
    vacancies_list = vacancies_block.find_all('div', {'class': vac_class})
   
    # определяем условие выхода из цикла (вакансий больше нет)
    if len(vacancies_list) == 0:
        break

    # итерируемся по списку, содержащем вакансии
    for vacancy in vacancies_list:
        salary = get_salary_sj(vacancy)

        
        # записываем полученные значения в словарь по каждой вакансии
        vacancy_data = {
            'url': get_url_sj(vacancy),
            'id': get_id_sj(vacancy),
            'title': get_title_sj(vacancy),
            'employer': get_employer_sj(vacancy),
            'min_salary': salary[0],
            'max_salary': salary[1],
            'currency': salary[2],
            'source': sj,
            'short_desc': get_description_sj(vacancy)
        }

        # пополняем основной словарь вакансий
        vacancies['results'].append(vacancy_data)

In [12]:
# обрабатываем случай, если по запросу ничего не найдено
hh_counter = 0
sj_counter = 0

for i in vacancies['results']:
    if i['source'] == hh:
        hh_counter += 1
    else:
        sj_counter +=1
        
if hh_counter == 0 and sj_counter == 0:
    print('По данному запросу ничего не найдено')

In [13]:
hh_counter, sj_counter

(93, 17)

In [14]:
print(len(vacancies['results']))

110


In [15]:
vacancies_df = pd.DataFrame(vacancies['results'])
vacancies_df

,url,id,title,employer,min_salary,max_salary,currency,source,short_desc
0,https://kolomna.hh.ru/vacancy/36527212?query=%...,hh_36527212,Прораб,PAY&WASH,50000.0,NaN,руб,https://hh.ru/,Заливке фундамента. Прокладке сетей (водопрово...
1,https://kolomna.hh.ru/vacancy/36433961?query=%...,hh_36433961,Руководитель строительного объекта,ООО Астра-Девелопмент,50000.0,50000.0,руб,https://hh.ru/,Организация и контроль выполнения работ на объ...
2,https://kolomna.hh.ru/vacancy/36196639?query=%...,hh_36196639,Прораб,ООО РОСМЕТ,70000.0,NaN,руб,https://hh.ru/,Максимально эффективное использование людских ...
3,https://kolomna.hh.ru/vacancy/36626925?query=%...,hh_36626925,Производитель работ,ООО АвтодорСтрой,60000.0,NaN,руб,https://hh.ru/,Организация работы бригады дорожных рабочих и ...
4,https://kolomna.hh.ru/vacancy/36646147?query=%...,hh_36646147,Мастер СМР (прораб),ООО АртВР-Строй,50000.0,NaN,руб,https://hh.ru/,Выполнение строительно-монтажных работ в соотв...
...,...,...,...,...,...,...,...,...,...
105,https://www.superjob.ru/vakansii/nachalnik-uch...,sj_32689472,Начальник участка (переезд в ЯНАО),РГС ГРУПП,100000.0,NaN,руб,https://www.superjob.ru,Организация строительного производства на учас...
106,https://www.superjob.ru/vakansii/nachalnik-mon...,sj_33387384,Начальник монтажного участка (м/к и т/т),Промстроймонтаж,150000.0,NaN,руб,https://www.superjob.ru,Организация работы линейного ИТР. Планирование...
107,https://www.superjob.ru/vakansii/proizvoditel-...,sj_33720883,Производитель работ / Старший производитель ра...,Промстроймонтаж,100000.0,110000.0,руб,https://www.superjob.ru,Организация работ по изготовлению металлоконст...
108,https://www.superjob.ru/vakansii/proizvoditel-...,sj_33684564,Производитель работ (улично-дорожная сеть Верх...,«Инженерно-строительный центр УГМК»,50000.0,55000.0,руб,https://www.superjob.ru,Обеспечение выполнения производственных задани...


In [16]:
vacancies_df.to_csv(f'{vacancies["query"]}.csv', index=False)